In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import random
import base64 # Import the base64 library for image encoding

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# These are the correct credentials from your mongodb-env.txt file
host = "nv-desktop-services.apporto.com"
port = 33561
username = "root"
password = "XhPBt3j862"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port)

# Read all data into a pandas DataFrame
df = pd.DataFrame.from_records(db.read({}))

# Drop the '_id' column as it can cause issues with Dash
if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# NEW METHOD TO LOAD THE IMAGE
# This reads the image file and encodes it to be embedded directly.
# This bypasses the 'assets' folder issue.
image_path = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_path, 'rb').read())

app.layout = html.Div([
    # Header with Logo and Title
    html.Center(html.A(
        # Use the encoded image string directly here
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height':'150px'}),
        href="https://www.snhu.edu"
    )),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.P("Written by: Nurlan Gasimzade")),
    html.Hr(),

    # Radio Buttons for Filtering
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'inline-block', 'margin': '0 10px'}
        )
    ),
    html.Hr(),

    # Interactive Data Table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),

    # Container for the two charts, side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to update the DataTable based on the radio button filter
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_table(filter_type):
    if filter_type == 'water':
        filtered_df = pd.DataFrame.from_records(db.find_water_rescue())
    elif filter_type == 'mountain':
        filtered_df = pd.DataFrame.from_records(db.find_mountain_rescue())
    elif filter_type == 'disaster':
        filtered_df = pd.DataFrame.from_records(db.find_disaster_rescue())
    else: # 'reset'
        filtered_df = pd.DataFrame.from_records(db.read({}))

    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'],inplace=True)

    return filtered_df.to_dict('records')

# Callback to update the Pie Chart based on the data in the DataTable
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [dcc.Graph(figure=px.pie(title='No Data to Display'))]
        
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Distribution of Breeds in Filtered Results')
        )
    ]

# Callback to update the Map based on the selected row in the DataTable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None or not index:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer()])]
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0]
    
    # Ensure the row index is valid
    if row >= len(dff):
        row = 0

    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns or dff.iloc[row]['location_lat'] is None:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[dl.TileLayer(), html.P("Location data not available for this entry.")])]
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the app 
selected_port = random.randint(8000, 8999)
app.run_server(mode='inline', port=selected_port)

Attempting to connect to the remote database...
Connected to MongoDB successfully!
